In [5]:
# === Latent Occupancy Field: Bootstrap Metaballs → Probability Isosurfaces (GLB) ===
# Requirements: numpy, pandas, scipy, scikit-image, trimesh
# Input: CSV with columns ['middle_x','middle_y','middle_z'] (or provide a DataFrame)
# Usage:
#   1) Set csv_path below (or pass df directly to run_latent_occupancy(df=...))
#   2) Run. GLBs will appear in out_latent_occ/.
# Notes:
#   - Tune n_boot / sample_frac / grid_base / sigma_vox / metaball_level for your data & speed.
#   - If 95% surface is empty, reduce prob_levels or increase n_boot.

import os, json
import numpy as np
import pandas as pd
from scipy.ndimage import gaussian_filter
from skimage.measure import marching_cubes
import trimesh

# -------------------------- USER SETTINGS --------------------------
csv_path      = "data/green_monkey/all_structure_files/chr1/12hrs/vacv/structure_12hrs_vacv_gene_info.csv"
out_dir       = "data/green_monkey/va_testing/out_latent_occ"
np.random.seed(7)

# Bootstrap + field settings
# n_boot          = 64          # ↑ for smoother probabilities (e.g., 128–256). 64 is quick-ish.
# sample_frac     = 0.8         # fraction of genes per bootstrap (with replacement)
# grid_base       = 128         # longest grid dimension (others scale by aspect). 160–192 if you have RAM.
# sigma_vox       = 2.0         # Gaussian blur in *voxels* (metaball smoothing)
# metaball_level  = 0.15        # iso on each smoothed field, as fraction of that field's max (0.10–0.25 typical)
# prob_levels     = [0.60, 0.80, 0.95]  # probability isosurfaces to export
# min_points      = 50          # safety: require at least this many points

n_boot         = 128        # smoother probabilities
sample_frac    = 0.90       # keep more genes per resample
grid_base      = 160        # finer grid reduces aliasing (watch RAM)
sigma_vox      = 3.0        # thicker local support → continuous lobes
metaball_level = 0.10       # ignored when thr_mode='quantile'
prob_levels    = [0.60, 0.80, 0.95]  # add 50% so you can see where occupancy starts
min_points      = 50          # safety: require at least this many points


# -------------------------- HELPERS --------------------------
def load_points(csv_path):
    df = pd.read_csv(csv_path)
    cols = ["middle_x","middle_y","middle_z"]
    if not all(c in df.columns for c in cols):
        raise ValueError(f"CSV must contain columns {cols}")
    pts = df[cols].dropna().values.astype(np.float32)
    return pts

def make_grid_from_bounds(pts, base=128, pad_frac=0.05):
    mins = pts.min(axis=0)
    maxs = pts.max(axis=0)
    ext  = maxs - mins
    pad  = pad_frac * float(ext.max())
    mins2 = mins - pad
    maxs2 = maxs + pad
    ext2  = maxs2 - mins2
    # proportional grid with longest axis == base
    scale = base / float(ext2.max() + 1e-8)
    dims  = np.ceil(ext2 * scale).astype(int)
    dims  = np.clip(dims, 32, 512)  # sanity bounds
    # histogramdd expects X,Y,Z ordering
    x_edges = np.linspace(mins2[0], maxs2[0], dims[0] + 1, dtype=np.float32)
    y_edges = np.linspace(mins2[1], maxs2[1], dims[1] + 1, dtype=np.float32)
    z_edges = np.linspace(mins2[2], maxs2[2], dims[2] + 1, dtype=np.float32)
    # voxel sizes
    vx = np.diff(x_edges)[0]; vy = np.diff(y_edges)[0]; vz = np.diff(z_edges)[0]
    origin_xyz = np.array([x_edges[0] + 0.5*vx, y_edges[0] + 0.5*vy, z_edges[0] + 0.5*vz], dtype=np.float32)
    return (x_edges, y_edges, z_edges), (vx, vy, vz), origin_xyz, dims

# def one_boot_mask(sample_pts, edges, sigma_vox, metaball_level):
#     # Histogram in X,Y,Z
#     H, _edges = np.histogramdd(sample_pts, bins=edges)  # returns (H, [x_edges,y_edges,z_edges])
#     H = H.astype(np.float32, copy=False)

#     # Gaussian smooth (metaball effect)
#     F = gaussian_filter(H, sigma=(sigma_vox, sigma_vox, sigma_vox), mode="constant")
#     F = F.astype(np.float32, copy=False)

#     thr = float(F.max()) * float(metaball_level)
#     if thr <= 0:
#         return None

#     mask_xyz = (F >= thr)

#     # Convert to Z,Y,X for marching_cubes compatibility later
#     mask_zyx = mask_xyz.transpose(2, 1, 0)
#     return mask_zyx

from scipy.ndimage import gaussian_filter, label

def one_boot_mask(sample_pts, edges, sigma_vox, metaball_level=None,
                  thr_mode="quantile", metaball_q=0.90, min_blob_vox=40):
    """
    Build a binary metaball mask for one bootstrap:
      - Smooths voxelized points (sigma_vox)
      - Thresholds by quantile (top (1-q) of field) or relative max
      - Drops tiny specks (< min_blob_vox voxels)
    """
    H, _ = np.histogramdd(sample_pts, bins=edges)
    H = H.astype(np.float32)

    F = gaussian_filter(H, sigma=(sigma_vox, sigma_vox, sigma_vox), mode="constant")
    F = F.astype(np.float32)

    if thr_mode == "quantile":
        vals = F[F > 0]
        if vals.size == 0:
            return None
        thr = float(np.quantile(vals, metaball_q))  # e.g., q=0.90 keeps top 10% of positive field
    else:  # "relative_max"
        use_level = 0.10 if metaball_level is None else metaball_level
        thr = float(F.max()) * float(use_level)

    if thr <= 0:
        return None

    mask_xyz = (F >= thr)

    # # de-speckle
    # lbl, nlab = label(mask_xyz)
    # if nlab > 0:
    #     sizes = np.bincount(lbl.ravel())
    #     keep_ids = np.where(sizes >= max(1, int(min_blob_vox)))[0]
    #     mask_xyz = np.isin(lbl, keep_ids)
    # de-speckle
    lbl, nlab = label(mask_xyz)
    if nlab > 0:
        sizes = np.bincount(lbl.ravel())
        # EXCLUDE background label 0
        keep_ids = np.where((np.arange(len(sizes)) > 0) & (sizes >= max(1, int(min_blob_vox))))[0]
        mask_xyz = np.isin(lbl, keep_ids)

    # convert to Z,Y,X for marching_cubes later
    return mask_xyz.transpose(2, 1, 0)


# def prob_isosurface(prob_zyx, level, spacing_zyx, origin_xyz):
#     if not (0.0 < level < 1.0):
#         return None
#     if (prob_zyx >= level).sum() == 0:
#         return None
#     verts_zyx, faces, _, _ = marching_cubes(prob_zyx, level=level, spacing=spacing_zyx)
#     # reorder to X,Y,Z and translate to world coords
#     verts_xyz = np.c_[verts_zyx[:,2], verts_zyx[:,1], verts_zyx[:,0]].astype(np.float32)
#     verts_xyz += origin_xyz  # shift to real-world coordinate system
#     mesh = trimesh.Trimesh(vertices=verts_xyz, faces=faces, process=False)
#     return mesh

def prob_isosurface(prob_zyx, level, spacing_zyx, origin_xyz, eps=1e-6):
    """
    Safe wrapper: if level is outside the actual data range, or if the
    surface is degenerate, return None.
    """
    # Ignore NaNs in range checks
    vmin = float(np.nanmin(prob_zyx))
    vmax = float(np.nanmax(prob_zyx))
    if not (vmin < level < vmax):   # must be strictly inside range
        return None

    # Must have both sides of the isovalue present somewhere
    ge = np.count_nonzero(prob_zyx >= level)
    lt = np.count_nonzero(prob_zyx <  level)
    if ge == 0 or lt == 0:
        return None

    # Nudge level away from exact plateaus
    lvl = float(np.clip(level, vmin + eps, vmax - eps))

    try:
        verts_zyx, faces, _, _ = marching_cubes(prob_zyx, level=lvl, spacing=spacing_zyx)
    except ValueError:
        return None

    if faces.size == 0:
        return None

    # reorder to X,Y,Z and translate to world coords
    verts_xyz = np.c_[verts_zyx[:,2], verts_zyx[:,1], verts_zyx[:,0]].astype(np.float32)
    verts_xyz += origin_xyz
    return trimesh.Trimesh(vertices=verts_xyz, faces=faces, process=False)


def ensure_outdir(path):
    os.makedirs(path, exist_ok=True)

# -------------------------- MAIN --------------------------
def run_latent_occupancy(csv=None, df=None):
    ensure_outdir(out_dir)

    # ---- load points ----
    if df is None and csv is None:
        raise ValueError("Provide csv=path or df=DataFrame.")
    pts = load_points(csv) if df is None else df[["middle_x","middle_y","middle_z"]].dropna().values.astype(np.float32)
    n = len(pts)
    if n < min_points:
        raise ValueError(f"Too few points ({n}). Need at least {min_points}.")

    # ---- grid ----
    edges, voxel_xyz, origin_xyz, dims_xyz = make_grid_from_bounds(pts, base=grid_base, pad_frac=0.05)
    x_edges, y_edges, z_edges = edges
    vx, vy, vz = voxel_xyz
    spacing_zyx = (vz, vy, vx)  # marching_cubes spacing order

    # ---- bootstrap occupancy count (Z,Y,X layout) ----
    occ_count = np.zeros((len(z_edges)-1, len(y_edges)-1, len(x_edges)-1), dtype=np.uint16)
    boot_k = max(1, int(round(sample_frac * n)))

    for b in range(n_boot):
        idx = np.random.choice(n, size=boot_k, replace=True)
        sample_pts = pts[idx]
        # mask_zyx = one_boot_mask(sample_pts, edges, sigma_vox=sigma_vox, metaball_level=metaball_level)
        mask_zyx = one_boot_mask(
            sample_pts, edges,
            sigma_vox=sigma_vox,
            metaball_level=metaball_level,   # still passed for fallback
            thr_mode="quantile",
            metaball_q=0.90,                 # try 0.88–0.92 if you need to tune
            min_blob_vox=40                  # raise to 80 if you want fewer small islands
        )

        if mask_zyx is None:
            continue
        occ_count += mask_zyx.astype(np.uint16)
        if (b+1) % max(1, n_boot//8) == 0:
            print(f"[{b+1}/{n_boot}]")

    # ---- probability field ----
    prob_zyx = (occ_count.astype(np.float32) / float(n_boot)).astype(np.float32)
    print(f"prob range: min={np.nanmin(prob_zyx):.3f}, max={np.nanmax(prob_zyx):.3f}")

    # ---- save volume & metadata ----
    np.save(os.path.join(out_dir, "latent_occ_prob.npy"), prob_zyx)
    meta = {
        "origin_xyz": origin_xyz.tolist(),
        "voxel_xyz": [float(vx), float(vy), float(vz)],
        "shape_zyx": list(map(int, prob_zyx.shape)),
        "n_boot": n_boot,
        "sample_frac": sample_frac,
        "grid_base": grid_base,
        "sigma_vox": sigma_vox,
        "metaball_level": metaball_level,
        "prob_levels": prob_levels
    }
    with open(os.path.join(out_dir, "latent_occ_meta.json"), "w") as f:
        json.dump(meta, f, indent=2)

    # ---- extract and export isosurfaces ----
    meshes = []
    colors = {
        0.60: [180,180,180,255],  # 60% light gray: speculative
        0.80: [120,120,220,255],  # 80% bluish: consistent
        0.95: [240,180,60,255],   # 95% gold: very stable core
    }
    label_map = {0.60:"p60", 0.80:"p80", 0.95:"p95"}

    for lvl in prob_levels:
        m = prob_isosurface(prob_zyx, level=float(lvl), spacing_zyx=spacing_zyx, origin_xyz=origin_xyz)
        if m is None or len(m.faces) == 0:
            print(f"⚠️  No surface found at {int(lvl*100)}% (try lowering level or increasing n_boot).")
            continue
        # color
        if colors.get(lvl):
            m.visual.face_colors = np.tile(np.array(colors[lvl], dtype=np.uint8), (len(m.faces), 1))
        tag = label_map.get(lvl, f"p{int(lvl*100)}")
        out_glb = os.path.join(out_dir, f"latent_occ_{tag}.glb")
        m.export(out_glb)
        print(f"✓ Saved {out_glb}  (faces={len(m.faces)})")
        meshes.append((tag, m))

    # ---- combined GLB with all meshes ----
    if meshes:
        scene = trimesh.Scene()
        for tag, m in meshes:
            scene.add_geometry(m, node_name=tag)
        combo_path = os.path.join(out_dir, "latent_occ_all.glb")
        scene.export(combo_path)
        print(f"✓ Saved layered GLB: {combo_path}")

    print("Done.")
    return prob_zyx, meta

# -------------------------- RUN --------------------------
if __name__ == "__main__":
    run_latent_occupancy(csv=csv_path)


[16/128]
[32/128]
[48/128]
[64/128]
[80/128]
[96/128]
[112/128]
[128/128]
prob range: min=0.000, max=1.000
✓ Saved data/green_monkey/va_testing/out_latent_occ/latent_occ_p60.glb  (faces=125768)
✓ Saved data/green_monkey/va_testing/out_latent_occ/latent_occ_p80.glb  (faces=91248)
✓ Saved data/green_monkey/va_testing/out_latent_occ/latent_occ_p95.glb  (faces=53194)
✓ Saved layered GLB: data/green_monkey/va_testing/out_latent_occ/latent_occ_all.glb
Done.


In [6]:
# ==== GLB Viewer (Plotly + Trimesh) ====
# pip install trimesh plotly
import numpy as np
import trimesh
import plotly.graph_objects as go
from pathlib import Path

def _mesh_to_traces(mesh, name=None, opacity=1.0, showlegend=True):
    """
    Convert a single Trimesh Trimesh() to a Plotly Mesh3d trace.
    """
    v = mesh.vertices
    f = mesh.faces
    trace = go.Mesh3d(
        x=v[:,0], y=v[:,1], z=v[:,2],
        i=f[:,0], j=f[:,1], k=f[:,2],
        name=name or "mesh",
        opacity=opacity,
        flatshading=True,
        showscale=False,
        lighting=dict(ambient=0.35, diffuse=0.7, specular=0.2, roughness=0.7, fresnel=0.2),
        lightposition=dict(x=1, y=1, z=1),
        hoverinfo="skip",
        showlegend=showlegend
    )
    return trace

def _scene_to_traces(scene, base_name, default_opacity=1.0):
    """
    Convert a Trimesh Scene() to Plotly traces using world-space meshes.
    Uses scene.dump(concatenate=False) so all node transforms are applied.
    """
    traces = []
    try:
        # Returns a list of Trimesh objects with transforms already applied
        meshes_world = scene.dump(concatenate=False)
    except Exception:
        # Fallback: apply transforms manually
        meshes_world = []
        for geom_name, mesh in scene.geometry.items():
            try:
                T = scene.graph.get_transform(geom_name)  # to world
            except Exception:
                T = np.eye(4)
            m = mesh.copy()
            m.apply_transform(T)
            meshes_world.append(m)

    for idx, tm in enumerate(meshes_world):
        traces.append(_mesh_to_traces(tm, name=f"{base_name}:{idx}", opacity=default_opacity))
    return traces


def show_glbs(glb_paths, opacities=None, titles=None):
    """
    Show one or more GLB files in a single interactive 3D view.
    - glb_paths: list of paths (or a single path)
    - opacities: list of floats (0-1), optional; defaults to 0.7 for all but last, 1.0 for last
    - titles: legend labels per file, optional
    """
    if isinstance(glb_paths, (str, Path)):
        glb_paths = [glb_paths]
    glb_paths = [Path(p) for p in glb_paths]

    if titles is None:
        titles = [p.stem for p in glb_paths]
    if opacities is None:
        opacities = [0.7]*max(0, len(glb_paths)-1) + ([1.0] if len(glb_paths) else [])

    fig = go.Figure()

    for p, alpha, title in zip(glb_paths, opacities, titles):
        # Force scene load, because a GLB may contain multiple nodes
        scene = trimesh.load(p.as_posix(), force='scene')
        if isinstance(scene, trimesh.Scene):
            traces = _scene_to_traces(scene, base_name=title, default_opacity=alpha)
        else:
            # sometimes a single mesh comes back
            traces = [_mesh_to_traces(scene, name=title, opacity=alpha)]
        for t in traces:
            fig.add_trace(t)

    # Aesthetics: turn off axes, set equal aspect, nice camera
    fig.update_scenes(
        xaxis_visible=False, yaxis_visible=False, zaxis_visible=False,
        aspectmode="data"
    )
    fig.update_layout(
        title="GLB Preview",
        scene_camera=dict(eye=dict(x=1.6, y=1.6, z=1.2)),
        legend=dict(itemsizing="constant"),
        margin=dict(l=0, r=0, t=30, b=0),
    )

    # fig.show()
    return fig

# ------------------ Examples ------------------
# 1) View one file:
# show_glbs("data/green_monkey/va_testing/out_latent_occ/latent_occ_all.glb")

# 2) View multiple surfaces together with different opacities:
# show_glbs([
#     "data/green_monkey/va_testing/out_latent_occ/latent_occ_p60.glb",
#     "data/green_monkey/va_testing/out_latent_occ/latent_occ_p80.glb",
#     "data/green_monkey/va_testing/out_latent_occ/latent_occ_p95.glb",
# ], opacities=[0.35, 0.55, 0.9], titles=["p60","p80","p95"])


In [1]:
# === One-shot viewer: shells (GLB) + raw gene points (Plotly) ===
# pip install trimesh plotly pandas

import numpy as np
import pandas as pd
import trimesh
import plotly.graph_objects as go
from pathlib import Path

def _mesh_traces_from_trimesh(mesh, name, opacity=1.0, color=None, wireframe=False, wire_width=1):
    v = mesh.vertices
    f = mesh.faces
    traces = [
        go.Mesh3d(
            x=v[:,0], y=v[:,1], z=v[:,2],
            i=f[:,0], j=f[:,1], k=f[:,2],
            name=name, opacity=opacity, color=color,
            flatshading=True, showscale=False,
            lighting=dict(ambient=0.35, diffuse=0.7, specular=0.2, roughness=0.7, fresnel=0.2),
            lightposition=dict(x=1, y=1, z=1),
            hoverinfo="skip", showlegend=True
        )
    ]
    if wireframe:
        # unique undirected edges
        edges = np.vstack([f[:,[0,1]], f[:,[1,2]], f[:,[2,0]]])
        edges = np.sort(edges, axis=1)
        edges = np.unique(edges, axis=0)
        # NaN separators to draw many segments in one trace
        x = np.empty(edges.shape[0]*3); x[:] = np.nan
        y = x.copy(); z = x.copy()
        x[0::3] = v[edges[:,0],0]; x[1::3] = v[edges[:,1],0]
        y[0::3] = v[edges[:,0],1]; y[1::3] = v[edges[:,1],1]
        z[0::3] = v[edges[:,0],2]; z[1::3] = v[edges[:,1],2]
        traces.append(go.Scatter3d(
            x=x, y=y, z=z, mode="lines",
            line=dict(width=wire_width), name=name+" (edges)",
            hoverinfo="skip", showlegend=False
        ))
    return traces

def _scene_meshes(scene):
    """Return a list of world-space Trimesh objects from a Trimesh scene or mesh."""
    if isinstance(scene, trimesh.Scene):
        try:
            return scene.dump(concatenate=False)
        except Exception:
            # fallback: apply transforms manually
            meshes = []
            for geom_name, mesh in scene.geometry.items():
                T = scene.graph.get_transform(geom_name)
                m = mesh.copy(); m.apply_transform(T)
                meshes.append(m)
            return meshes
    else:
        return [scene]

def _load_gene_points(csv_path, cols=("middle_x","middle_y","middle_z")):
    df = pd.read_csv(csv_path)
    pts = df[list(cols)].dropna().values.astype(np.float32)
    return pts

def show_shells_and_points(
    shell_paths,                      # list[str|Path] e.g. [p60.glb, p80.glb, p95.glb]
    shell_titles=None,                # ["p60","p80","p95"]
    shell_colors=None,                # ["#9db4ff","#ffb085","#2e8b57"]
    shell_opacities=None,             # [0.30, 0.55, 0.90]
    wireframe=True, wire_width=1,
    csv_path=None,                    # points CSV (optional)
    point_size=2, point_color="black", point_opacity=0.45, subsample=None,
    camera_eye=(1.6,1.6,1.2),
    show=True
):
    # Normalize inputs
    shell_paths = [Path(p) for p in (shell_paths if isinstance(shell_paths, (list, tuple)) else [shell_paths])]
    n = len(shell_paths)
    if shell_titles is None:   shell_titles = [p.stem for p in shell_paths]
    if shell_colors is None:   shell_colors = [None]*n
    if shell_opacities is None:
        shell_opacities = [0.7]*max(0, n-1) + ([1.0] if n else [])

    fig = go.Figure()

    # Add shells
    for p, title, color, alpha in zip(shell_paths, shell_titles, shell_colors, shell_opacities):
        scene = trimesh.load(p.as_posix(), force='scene')
        for idx, mesh in enumerate(_scene_meshes(scene)):
            for tr in _mesh_traces_from_trimesh(
                mesh, name=f"{title}" if n==1 else f"{title}:{idx}",
                opacity=alpha, color=color, wireframe=wireframe, wire_width=wire_width
            ):
                fig.add_trace(tr)

    # Add points (optional)
    if csv_path is not None:
        pts = _load_gene_points(csv_path)
        if subsample is not None and subsample < len(pts):
            idx = np.random.choice(len(pts), subsample, replace=False)
            pts = pts[idx]
        fig.add_trace(go.Scatter3d(
            x=pts[:,0], y=pts[:,1], z=pts[:,2],
            mode="markers",
            marker=dict(size=point_size, opacity=point_opacity, color=point_color),
            name="genes", hoverinfo="skip", showlegend=True
        ))

    # Layout
    fig.update_scenes(xaxis_visible=False, yaxis_visible=False, zaxis_visible=False, aspectmode="data")
    fig.update_layout(
        title="Genes over Latent Occupancy Shells" if csv_path else "Latent Occupancy Shells",
        scene_camera=dict(eye=dict(x=camera_eye[0], y=camera_eye[1], z=camera_eye[2])),
        legend=dict(itemsizing="constant"),
        margin=dict(l=0, r=0, t=30, b=0),
    )
    if show:
        fig.show()
        return
    return fig



csv_path = "data/green_monkey/all_structure_files/chr1/12hrs/vacv/structure_12hrs_vacv_gene_info.csv"
shells = [
    "data/green_monkey/va_testing/out_latent_occ/latent_occ_p60.glb",
    "data/green_monkey/va_testing/out_latent_occ/latent_occ_p80.glb",
    "data/green_monkey/va_testing/out_latent_occ/latent_occ_p95.glb",
]

# show_shells_and_points(
#     shells,
#     shell_titles=["p60","p80","p95"],
#     shell_colors=["#9db4ff", "#ffb085", "#2e8b57"],
#     shell_opacities=[0.30, 0.55, 0.90],
#     wireframe=False,              # 🔹 disables edge overlays
#     csv_path=csv_path,
#     point_size=2, point_color="black", point_opacity=0.45,
#     subsample=None,
#     camera_eye=(1.6,1.6,1.2),
#     show=True
# )

